### And now - Week 3 Day 3

## AutoGen Core

Something a little different.

This is agnostic to the underlying Agent framework

You can use AutoGen AgentChat, or you can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

### The fundamental principle

Autogen Core decouples an agent's logic from how messages are delivered.  
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.

The communication infrastructure is called a Runtime.

There are 2 types: **Standalone** and **Distributed**.

Today we will use a standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.

Tomorrow we'll briefly look at a Distributed runtime.


In [3]:
# Import dataclass for easily creating classes that store data
from dataclasses import dataclass

# Import core types and utilities from AutoGen Core:
# AgentId: Used to uniquely identify an agent in the framework
# MessageContext: Holds context information about the current message being processed
# RoutedAgent: Base class for defining custom agents, with support for routing and handling messages
# message_handler: Decorator to mark agent methods to automatically handle specific message types
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler

# Import the runtime implementation that executes agents and delivers messages in a local, single-threaded environment
from autogen_core import SingleThreadedAgentRuntime

# Import AssistantAgent, a prebuilt agent from the AgentChat module supporting conversational assistant logic
from autogen_agentchat.agents import AssistantAgent

# Import TextMessage, the standard message type used to represent text content between agents in AgentChat
from autogen_agentchat.messages import TextMessage

# Import OpenAIChatCompletionClient, a client class for interacting with OpenAI's chat completion APIs for generating responses
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)


True

### First we define our Message object

Whatever structure we want for messages in our Agent framework.

In [4]:
# We'll use a minimal and clear message structure for our agent framework.
# The @dataclass decorator allows us to easily define a lightweight class that simply holds data,
# making our Message object immutable (unless otherwise specified) and auto-generating useful methods 
# like __init__, __repr__, and __eq__ behind the scenes.
# Here, Message will just encapsulate the content of an exchange between agents.
@dataclass
class Message:
    content: str


### Now we define our Agent

A subclass of RoutedAgent.

Every Agent has an **Agent ID** which has 2 components:  
`agent.id.type` describes the kind of agent it is  
`agent.id.key` gives it its unique identifier

Any method with the `@message_handler` decorated will have the opportunity to receive messages.


In [5]:
# We define our class SimpleAgent that is a sub-class of RoutedAgent
class SimpleAgent(RoutedAgent):
    # In Python, the constructor method is always called __init__.
    # It is called automatically when a new instance of the class is created.
    # The first parameter of every instance method (including __init__) must be 'self', which refers to the instance itself.
    # Constructors can take additional arguments after 'self' to initialize object state.
    # Inside __init__, we usually initialize attributes or call the parent constructor with super().__init__()
    def __init__(self) -> None:
        # Here we call the parent class's constructor with a specific argument ("Simple").
        # This argument typically sets the agent's type, which is important for agent identification
        # and routing within the framework. By providing "Simple", we ensure this agent is recognized
        # as a SimpleAgent throughout the system, allowing for correct handling, message dispatching,
        # and potential differentiation from other agent types. Passing this value to the parent 
        # constructor lets the superclass logic manage shared setup behaviors while using this subclass's identity.
        super().__init__("Simple")

    # Here we use the @message_handler decorator (imported from AutoGen Core), which enables this method to automatically receive messages
    # intended for this agent, according to the agent framework's routing rules.
    @message_handler
    # TO BE NOTED: This method does not invoke an LLM or any external service; it simply processes the incoming Message
    # and returns a new instance of our class Message with a simple string response constructed using the agent's id and input content.
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")
        

### OK let's create a Standalone runtime and register our agent type

In [ ]:
# Here we create a standalone runtime and we register our SimpleAgent class (the one withourt LLM)
runtime = SingleThreadedAgentRuntime()
# The lambda parameter is used here to provide a factory function that creates a new instance of SimpleAgent each time the agent is needed by the runtime.
# This ensures that each agent has its own fresh state and avoids using a pre-created, potentially shared, instance.
await SimpleAgent.register(
    runtime, "simple_agent", 
    lambda: SimpleAgent()
)

AgentType(type='simple_agent')

### Alright! Let's start a runtime and send a message

In [7]:
# Here we start the runtime
runtime.start()

In [ ]:
# Create an AgentId object with type "simple_agent" and key "default" (REMEMBER: AgentId needs 2 components: type and key)
# This identifies the agent we want to send a message to within the runtime system
agent_id = AgentId(
    type="simple_agent", 
    key="default"
)

# Send a message "Well hi there!" to the agent identified above, using the runtime,
# and wait for the agent's response asynchronously. 
# Message() creates a message object to encapsulate the content for dispatch.
response = await runtime.send_message(
    message=Message("Well hi there!"), 
    recipient=agent_id
)

# Output the response content received from the agent to the console, 
# prefixing it with ">>> " for clarity in output display.
# It will print "You said 'Well hi there!' and I disagree"
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well hi there!' and I disagree.


In [ ]:
# Here we stop and close the single-threated runtime
await runtime.stop()
await runtime.close()

### OK Now let's do something more interesting

We'll use an AgentChat Assistant!

In [ ]:
# Here we build a more complex agent with LLM call
class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        # IMPORTANT: 
        # In this agent class, we want to give our agent the power to interact with a large language model (LLM) such as OpenAI's GPT-4o-mini.
        # To achieve this, we set up a "model client" that knows how to send and receive messages from the LLM.
        # We then create an AssistantAgent (from the agentchat system) and store it privately on this agent as `self._delegate`.
        # This means: whenever we want our MyLLMAgent to answer, it will internally use the delegate to call the LLM, 
        # letting our agent inherit powerful language capabilities without duplicating LLM-handling code.
        # In summary: self._delegate wraps an LLM-powered AssistantAgent that MyLLMAgent can use to answer complex queries.
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name="LLMAgent", model_client=model_client) # This method will be use to call the LLM thanks to AssistanAgent() class

    # message_handler is a wrapper function to provide my funtion handle_my_message_type the capability to receive messages
    @message_handler
    # Parameters in Python functions are annotated with `:` to specify their types (type hints), not with `=` which is used for default values.
    # The colon `:` after each parameter indicates its expected type, improving code readability and enabling type checking tools.
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        # Print out a message indicating that this agent type received an incoming message with its content shown
        print(f"{self.id.type} received message: {message.content}")
        # Convert the incoming message (from generic Message) into a TextMessage, indicating it's from the "user"
        # TexMessage is an AutoGen Chat class for handle text messages types
        text_message = TextMessage(content=message.content, source="user")
        # Asynchronously pass the TextMessage to the underlying LLM-powered delegate (AssistantAgent) to get a response
        # Here we use the ._delegate method of our class MyLLMAgent to pass the message to the LLM
        response = await self._delegate.on_messages(messages=[text_message], cancellation_token=ctx.cancellation_token)
        # Extract the content of the reply from the response's chat_message attribute
        reply = response.chat_message.content
        # Print out what the agent is about to respond with, for debugging and clarity
        print(f"{self.id.type} responded: {reply}")
        # Return a new instance of Message object for the runtime to handle
        # This coul be the input for another agent in case of agent workflow
        return Message(content=reply)
    


In [12]:
from autogen_core import SingleThreadedAgentRuntime  # Import the SingleThreadedAgentRuntime which manages agent execution in a single thread

runtime = SingleThreadedAgentRuntime()  # Instantiate a new single-threaded runtime environment for managing agent interactions

# Register the SimpleAgent with the runtime:
# - Uses the name "simple_agent" to identify this agent within the runtime.
# - The lambda returns a new instance of SimpleAgent each time the agent is requested or spawned.
await SimpleAgent.register(runtime=runtime, type="simple_agent", factory=lambda: SimpleAgent())

# Register the MyLLMAgent with the runtime:
# - Uses the name "LLMAgent" to identify this LLM-capable agent type.
# - The lambda returns a new instance of MyLLMAgent, ensuring each request gets a fresh agent.
await MyLLMAgent.register(runtime=runtime, type="LLMAgent", factory=lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [13]:
runtime.start()  # Start processing messages in the background.
response = await runtime.send_message(message=Message("Hi there!"), recipient=AgentId(type="LLMAgent", key="default"))
print(">>>", response.content) # the LLM agent returns: Hello! How can I assist you today?
response =  await runtime.send_message(message=Message(response.content), recipient=AgentId(type="simple_agent", key="default"))
print(">>>", response.content) # the simple agent returns: This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
response = await runtime.send_message(message=Message(response.content), recipient=AgentId(type="LLMAgent", key="default"))
# TO BE NOTED: I added these additional lines to better undestand how AutoGen Core operates in managing messages workflow
response = await runtime.send_message(message=Message("Which is the capital of Italy?"), recipient=AgentId("LLMAgent", "default"))
print(">>>", response.content)

Task was destroyed but it is pending!
task: <Task pending name='Task-8' coro=<RunContext._run() running at /Users/federico.tognetti/Desktop/AgenticAIEngineering/agents/.venv/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py:110> wait_for=<Future pending cb=[Task.task_wakeup()]>>


LLMAgent received message: Hi there!
LLMAgent responded: Hello! How can I assist you today?
>>> Hello! How can I assist you today?
>>> This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
LLMAgent received message: This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
LLMAgent responded: I appreciate your feedback! How would you prefer I assist you?
LLMAgent received message: Which is the capital of Italy?
LLMAgent responded: The capital of Italy is Rome.
>>> The capital of Italy is Rome.


In [14]:
# Here we stop and close the single-threaded runtime
await runtime.stop()
await runtime.close()

### OK now let's show this at work - let's have 3 agents interact!

In [16]:
from autogen_ext.models.ollama import OllamaChatCompletionClient

# Palyer 1 delegates to gpt-4o-mini
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
# Player 2 delegates to llama3.2
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [26]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name=name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId(type="player1", key="default")
        inner_2 = AgentId(type="player2", key="default")
        response1 = await self.send_message(message=message, recipient=inner_1)
        response2 = await self.send_message(message=message, recipient=inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        # Tha following message is an instance of AutoGen chat
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)


In [27]:
runtime = SingleThreadedAgentRuntime()

await Player1Agent.register(
    runtime=runtime,
    type="player1",
    factory=lambda: Player1Agent("player1")
)

await Player2Agent.register(
    runtime=runtime,
    type="player2",
    factory=lambda: Player2Agent("player2")
)

await RockPaperScissorsAgent.register(
    runtime=runtime,
    type="rock_paper_scissors",
    factory=lambda: RockPaperScissorsAgent("rock_paper_scissors")
)
runtime.start()

In [28]:
agent_id = AgentId(
    type="rock_paper_scissors", 
    key="default"
)
message = Message(content="go")
response = await runtime.send_message(message=message, recipient=agent_id)
print(response.content)

Player 1: rock
Player 2: scissors
Player 1 wins because rock beats scissors. TERMINATE


In [25]:
await runtime.stop()
await runtime.close()